In [1]:
import datetime

from csnake import CodeWriter
import platform
import psutil
import logging
from is_anchored import gen_IS_anchored_program, gen_IS_anchored_program_block
from os_anchored import gen_OS_anchored_program, gen_OS_anchored_program_block
from ws_anchored import gen_WS_anchored_program, gen_WS_anchored_program_block
import os
import json
import math

In [2]:
def generate_whole_program(precision, vec_len, anchor_stationarity, aux_stationarity, with_blocking: bool = False, block_scheme = None):

    if with_blocking:
        if anchor_stationarity == "IS":
            gen_IS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        elif anchor_stationarity == "WS":
            gen_WS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        elif anchor_stationarity == "OS":
            gen_IS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        else:
            raise ValueError("anchoring stationarity not found")
            
    else:
        if anchor_stationarity == "IS":
            gen_IS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        elif anchor_stationarity == "WS":
            gen_WS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        elif anchor_stationarity == "OS":
            gen_IS_anchored_program(cw, precision, vec_len, fh, fw, aux_stationarity,stride)
        else:
            raise ValueError("anchoring stationarity not found")

In [8]:
root_dir = "/home/zhouzikai/nn_ext_dataflows/gen_programs/noblock/src/"
vl_list = [128, 256,512]

for filter_size in range(3,6):
    for stride in range(1,2):
        for vl in vl_list:
            num_allocatable_vec = (32 // (vl // 128)) - 3
            max_io_in_out_num = filter_size * (filter_size - stride) // (vl//128)
            max_io_weight_num = filter_size * filter_size // (vl //128)

            max_ws_in_out_num = 32 // (vl // 128)

            # Generate IS anchored
            for no in range(0, num_allocatable_vec+1):
                nw = num_allocatable_vec - no
                if no > max_io_in_out_num or nw > max_io_weight_num:
                    continue

                cw = CodeWriter()
                gen_IS_anchored_program(cw, 1, vl, filter_size, filter_size, {"WS":nw, "OS":no},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_bin_stride_"+str(stride)+"_fs_"+str(filter_size)+"_is_ws_"+str(nw)+"_os_"+str(no)+".cpp")
                del cw 

                cw = CodeWriter()
                gen_IS_anchored_program(cw, 8, vl, filter_size, filter_size, {"WS":nw, "OS":no},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_8bt_stride_"+str(stride)+"_fs_"+str(filter_size)+"_is_ws_"+str(nw)+"_os_"+str(no)+".cpp")
                del cw 

            # Generate OS anchored
            for ni in range(0,num_allocatable_vec+1):
                nw = num_allocatable_vec - ni
                if ni > max_io_in_out_num or nw > max_io_weight_num:
                    continue
                cw = CodeWriter()
                gen_OS_anchored_program(cw, 1, vl, filter_size, filter_size, {"WS":nw, "IS":ni},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_bin_stride_"+str(stride)+"_fs_"+str(filter_size)+"_os_ws_"+str(nw)+"_is_"+str(ni)+".cpp")
                del cw 

                cw = CodeWriter()
                gen_OS_anchored_program(cw, 8, vl, filter_size, filter_size, {"WS":nw, "IS":ni},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_8bt_stride_"+str(stride)+"_fs_"+str(filter_size)+"_os_ws_"+str(nw)+"_is_"+str(ni)+".cpp")
                del cw 

            # Generate WS anchored
            for ni in range(0,num_allocatable_vec+1):
                no = num_allocatable_vec - ni
                cw = CodeWriter()
                gen_WS_anchored_program(cw, 1, vl, filter_size, filter_size, {"OS":no, "IS":ni},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_bin_stride_"+str(stride)+"_fs_"+str(filter_size)+"_ws_os_"+str(no)+"_is_"+str(ni)+".cpp")
                del cw 

                cw = CodeWriter()
                gen_WS_anchored_program(cw, 8, vl, filter_size, filter_size, {"OS":no, "IS":ni},stride)
                cw.write_to_file(root_dir+"vl"+str(vl)+"_8bt_stride_"+str(stride)+"_fs_"+str(filter_size)+"_ws_os_"+str(no)+"_is_"+str(ni)+".cpp")
                del cw 